In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from inputs.constants import DATA_PATH

In [ ]:
df = pd.read_csv(DATA_PATH / 'aus_vax_data.csv', index_col=424)
df.index = pd.to_datetime(df.index, infer_datetime_format=True)

In [ ]:
age_16_3dose = [
    col for col in df.columns if 
    'National' in col and 
    any([s in col for s in ['who have received 3 doses', 'with 3 or more doses', 'with more than two doses', 'with more than 2 doses']]) and 
    any([s in col for s in ['16', '18']]) and
    not any([s in col for s in ['increase', 'Percentage', 'Indigenous']])
]
age_16_4dose = [
    col for col in df.columns if
    'National' in col and
    any([s in col for s in ['Winter Boosters number', 'who have received 4 doses', 'Fourth dose number']]) and
    not any([s in col for s in ['30', '65', 'increase']])
]
age_12_15_2dose = [
    col for col in df.columns if
    '12-15' in col and
    any([s in col for s in ['National', 'Age group']]) and
    any([s in col for s in ['2 doses', 'fully vaccinated']]) and
    not any([s in col for s in ['Indigenous', 'Population', '- F -', '- M -']])
]
age_5_11_2dose = [
    col for col in df.columns if
    'National' in col and
    '5-11' in col and
    any([s in col for s in ['2 doses', 'fully vaccinated']])
]
col_masks = {
    'Ages 16+, at least three doses': age_16_3dose,
    'Ages 16+, at least four doses': age_16_4dose,
    'Ages 12 to 15, at least two doses': age_12_15_2dose,
    'Ages 5 to 11, at least two doses': age_5_11_2dose,
}

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles =list(col_masks.keys()))
for m, mask in enumerate(col_masks):
    col = m % 2 + 1
    row = int(np.floor(m / 2)) + 1
    fig.add_traces(px.line(df[col_masks[mask]]).data, rows=row, cols=col)
fig.update_layout(height=600, showlegend=False)

In [ ]:
# Create a new dataframe that combines 16+ 3-dose and 4-dose coverage to create a  booster over time
df_booster = df[['National - Number of people 16 and over who have received 3 doses',
                    'National - 16 and over with 3 or more doses',
                    'National - 18 and over with more than 2 doses',
                    'National - Number of people 16 and over with more than two doses',
                    'National - 16 and over with more than 2 doses',
                    'National - Winter Boosters number- daily update',
                    'National - Winter Boosters number- weekly update',
                    'National - Number of people 16 and over who have received 4 doses',
                    'National - Fourth dose number- daily update']]

In [ ]:
df_booster['Adult booster doses'] = df_booster.loc[:,['National - Number of people 16 and over who have received 3 doses',
                    'National - 16 and over with 3 or more doses',
                    'National - 18 and over with more than 2 doses',
                    'National - Number of people 16 and over with more than two doses',
                    'National - 16 and over with more than 2 doses',
                    'National - Winter Boosters number- daily update',
                    'National - Winter Boosters number- weekly update',
                    'National - Number of people 16 and over who have received 4 doses',
                    'National - Fourth dose number- daily update']].sum(axis=1)

In [ ]:
df_booster['Adult booster smooth'] = df_booster.loc[:, 'Adult booster doses'].rolling(14).mean()

In [ ]:
px.line(df_booster['Adult booster smooth'])

In [ ]:
# visualise 2-dose coverage
px.line(df[[c for c in df.columns if 'Number of people fully vaccinated' in c]].dropna())